In [1]:
import numpy as np
import pyDR as DR
from pyDR.Project import Project
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import pickle

In [2]:
ile_resn = [3, 13, 23, 30, 36, 44, 61]
test_list = ["test_001", "test_002", "test_003", "test_004"]

In [3]:
def get_frame_analysis(test):
    tf=200000
    molsys=DR.MolSys('/mnt/c/Users/thier/Desktop/1ubq_frame.pdb', '/home/mlehner/1ubq/'+test+'/run_001/traj_4us_PBC_fit.xtc',tf=tf)
    select=DR.MolSelect(molsys)
    frames=list()
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile_resn,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':0})

    "Frames with post-process smoothing"
    frames.append({'Type':'hops_3site','Nuc':'ivl', 'resids':ile_resn,'sigma':5})
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile_resn,'sigma':5})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':50})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':50})
    select.select_bond(Nuc='ivl', resids=ile_resn)
    fr_obj=DR.Frames.FrameObj(select)
    fr_obj.tensor_frame(sel1=1,sel2=2)
    for f in frames:fr_obj.new_frame(**f)
    fr_obj.load_frames(n=-1)
    fr_obj.post_process()
    include=np.zeros([3,9],dtype=bool)
    include[0][:3]=True  
    include[1][[4,6,8]]=True
    include[2][3:]=True
    t=np.arange(tf>>1)*.005
    data=fr_obj.frames2data(mode='full')
    proj=Project('/home/mlehner/pyDR/pyDR/test_project'+test+'.nosync',create=True)
    for d in data:proj.append_data(d)
    proj[0].detect.r_no_opt(10)
    proj.fit(bounds=False)
    proj['no_opt'][0].detect.r_auto(5)
    proj['no_opt'].fit()
    return fr_obj.frames2ct(include=include[-1], mode='full')
    

In [4]:
for test in test_list:
    out = get_frame_analysis(test)
    pickle.dump(out, open("/home/mlehner/pyDR/out_figures/ile/"+test+"/out2.p", 'wb'))

Loading Ref. Frames: |██████████████████████████████████████████████████| 100.0% Complete
Completed
Applying default post processing (only active for frames that define their own post processing)
Fitted 12 data objects
Fitted 12 data objects
Loading Ref. Frames: |██████████████████████████████████████████████████| 100.0% Complete
Completed
Applying default post processing (only active for frames that define their own post processing)
Fitted 12 data objects
Fitted 12 data objects
Loading Ref. Frames: |██████████████████████████████████████████████████| 100.0% Complete
Completed
Applying default post processing (only active for frames that define their own post processing)
Fitted 12 data objects
Fitted 12 data objects
Loading Ref. Frames: |██████████████████████████████████████████████████| 100.0% Complete
Completed
Applying default post processing (only active for frames that define their own post processing)
Fitted 12 data objects
Fitted 12 data objects


In [7]:
out1=pickle.load(open("/home/mlehner/pyDR/out_figures/ile/test_001/out2.p", 'rb'))

In [8]:
out1["ct_prod"].shape

(42, 200000)